In [27]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib
import yfinance as yfin
import constants as cs

%reload_ext autoreload
%autoreload 2



In [33]:

def getdata(data_startdate : str, data_enddate: str):
    data = yfin.download(['LONN.SW', 'SIKA.SW'], period= 'max')['Adj Close']
    data = data.loc[str(data_startdate) : str(data_enddate)]
    return data 

data = getdata(data_startdate=cs.data_startdate, data_enddate=cs.data_enddate)

[*********************100%***********************]  2 of 2 completed


In [29]:
def rolling_window(r: pd.DataFrame, window_len: int, window_start: pd.Timestamp, data_enddate: pd.Timestamp):
    n_periods = r.shape[0]
    windows = [(window_start + pd.offsets.BDay(start), window_start + pd.offsets.BDay(start + window_len)) for start in range(n_periods - window_len)]
    return windows


rolling_window(r = data, window_len=60, window_start=cs.sim_start_date, data_enddate=cs.data_enddate)[1][0]

Timestamp('2023-05-01 00:00:00')

In [32]:
def multi_asset_gbm(data:pd.DataFrame, sim_start_date: pd.Timestamp, 
                         windows: list) -> pd.DataFrame:
    # formatting historical data
    try: 
        for window in range(len(windows)):
            s0_vector = data.loc[str(windows[window][0])]
            print(f"S_0_vector: {s0_vector}")
    except Exception as e:
            raise Exception("Error at wrangling historical data.")
    
multi_asset_gbm(data = data, sim_start_date=cs.sim_start_date, windows = rolling_window(r = data, window_len=60, window_start=cs.sim_start_date, data_enddate=cs.data_enddate) )
    


S_0_vector: Ticker
LONN.SW    546.263306
SIKA.SW    242.368103
Name: 2023-04-28 00:00:00+00:00, dtype: float64


Exception: Error at wrangling historical data.